<a href="https://colab.research.google.com/github/RohannnCodes/STSCI5050-Final-Project/blob/main/winners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nba_api
from nba_api.stats.endpoints import PlayByPlayV2
from nba_api.stats.endpoints import LeagueGameFinder
from nba_api.stats.endpoints import BoxScoreTraditionalV3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ot_starting_lineups_2016_17.csv to ot_starting_lineups_2016_17.csv
Saving ot_starting_lineups_2017_18.csv to ot_starting_lineups_2017_18.csv
Saving ot_starting_lineups_2018_19.csv to ot_starting_lineups_2018_19.csv
Saving ot_starting_lineups_2019_20.csv to ot_starting_lineups_2019_20.csv


In [ ]:
df_2016 = pd.read_csv("/content/ot_starting_lineups_2016_17.csv")
df_2017 = pd.read_csv("/content/ot_starting_lineups_2017_18.csv")
df_2018 = pd.read_csv("/content/ot_starting_lineups_2018_19.csv")
df_2019 = pd.read_csv("/content/ot_starting_lineups_2019_20.csv")

In [ ]:
merged_df = pd.concat([df_2016, df_2017, df_2018], ignore_index=True)
merged_df.to_csv("merged_output.csv", index=False)

merged_df.head()

,Unnamed: 0,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,season,PLAYER_NAME_CLEAN
0,0.0,21601205,1610612748,MIA,203186,Willie Reed,2016-17,Willie Reed
1,1.0,21601205,1610612739,CLE,1627770,Kay Felder,2016-17,Kay Felder
2,2.0,21601205,1610612748,MIA,1626196,Josh Richardson,2016-17,Josh Richardson
3,3.0,21601205,1610612739,CLE,101114,Deron Williams,2016-17,Deron Williams
4,4.0,21601205,1610612748,MIA,201609,Goran Dragic,2016-17,Goran Dragic


In [ ]:
unique_game_ids = merged_df["GAME_ID"].unique()

In [ ]:
from nba_api.stats.endpoints import BoxScoreTraditionalV3
import pandas as pd
import time

def get_winner(game_id):
    # Pull raw JSON
    box = BoxScoreTraditionalV3(game_id=game_id).get_dict()
    game = box["boxScoreTraditional"]

    # Extract team objects
    home = game["homeTeam"]
    away = game["awayTeam"]

    # Extract info
    home_team = home["teamTricode"]
    away_team = away["teamTricode"]

    home_team_id = home["teamId"]
    away_team_id = away["teamId"]

    home_pts = home["statistics"]["points"]
    away_pts = away["statistics"]["points"]

    # Determine winner
    if home_pts > away_pts:
        winner_side = "HOME"
        winner_team = home_team
        winner_team_id = home_team_id
    else:
        winner_side = "AWAY"
        winner_team = away_team
        winner_team_id = away_team_id

    return {
        "GAME_ID": game_id,
        "WINNER_SIDE": winner_side,
        "WINNER_TEAM": winner_team,
        "WINNER_TEAM_ID": winner_team_id,
        "HOME_TEAM": home_team,
        "HOME_TEAM_ID": home_team_id
    }

In [ ]:
winners = []

for gid in unique_game_ids:
    print(f"Processing game: {gid}")
    try:
        info = get_winner(str(gid).zfill(10))
        winners.append(info)
    except Exception as e:
        print(f"Error for {gid}: {e}")
    time.sleep(1.0)   # avoid NBA API rate limits

# Convert to DataFrame
winners_df = pd.DataFrame(winners)

Processing game: 21601205
Processing game: 21601197
Processing game: 21601161
Processing game: 21601150
Processing game: 21601130
Processing game: 21601112
Processing game: 21601080
Processing game: 21601051
Processing game: 21601045
Processing game: 21600976
Processing game: 21600983
Processing game: 21600972
Processing game: 21600963
Processing game: 21600931
Processing game: 21600925
Processing game: 21600917
Processing game: 21600890
Processing game: 21600881
Processing game: 21600859
Processing game: 21600851
Processing game: 21600830
Processing game: 21600797
Processing game: 21600784
Processing game: 21600769
Processing game: 21600764
Processing game: 21600749
Processing game: 21600725
Processing game: 21600720
Processing game: 21600711
Processing game: 21600706
Processing game: 21600696
Processing game: 21600680
Processing game: 21600657
Processing game: 21600650
Processing game: 21600648
Processing game: 21600559
Processing game: 21600554
Processing game: 21600550
Processing g

In [ ]:
winners_df.head(10)

,GAME_ID,WINNER_SIDE,WINNER_TEAM,WINNER_TEAM_ID,HOME_TEAM,HOME_TEAM_ID
0,0021601205,HOME,MIA,1610612748,MIA,1610612748
1,0021601197,HOME,ATL,1610612737,ATL,1610612737
2,0021601161,HOME,SAS,1610612759,SAS,1610612759
3,0021601150,HOME,CLE,1610612739,CLE,1610612739
4,0021601130,HOME,MIL,1610612749,MIL,1610612749
5,0021601112,AWAY,OKC,1610612760,ORL,1610612753
6,0021601080,HOME,LAL,1610612747,LAL,1610612747
7,0021601051,HOME,TOR,1610612761,TOR,1610612761
8,0021601045,HOME,ORL,1610612753,ORL,1610612753
9,0021600976,AWAY,NOP,1610612740,CHA,1610612766


In [ ]:
winners_df.to_csv("merged_winners.csv")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2016-17-rapm.json to 2016-17-rapm (1).json
Saving 2017-18-rapm.json to 2017-18-rapm (1).json
Saving 2018-19-rapm.json to 2018-19-rapm (1).json


In [ ]:
import json
import pandas as pd
import os

files = [
    "/content/2016-17-rapm.json",
    "/content/2017-18-rapm.json",
    "/content/2018-19-rapm.json"
]

for file in files:
    # Load JSON file
    with open(file, "r") as f:
        raw = json.load(f)

    # Define column names
    cols = ["Rank", "Player", "Team", "Poss", "ORAPM", "DRAPM", "RAPM"]

    # Create DataFrame
    df = pd.DataFrame(raw["data"], columns=cols)

    # Extract year from file name → "2016-17"
    base_name = os.path.basename(file)          # "2016-17-rapm.json"
    year = base_name.split("-rapm")[0]          # "2016-17"

    # Build output file name
    out_name = f"rapm_{year}.csv"               # "rapm_2016-17.csv"

    # Save to CSV
    df.to_csv(out_name, index=False)

In [ ]:
df_players = pd.read_csv("merged_output.csv", index_col=0)

rapm_2016 = pd.read_csv("/content/rapm_2016-17.csv", index_col=0)
rapm_2017 = pd.read_csv("/content/rapm_2017-18.csv", index_col=0)
rapm_2018 = pd.read_csv("/content/rapm_2018-19.csv", index_col=0)

df_players.head(1980)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,season,PLAYER_NAME_CLEAN
Unnamed: 0,,,,,,,
0.0,21601205,1610612748,MIA,203186,Willie Reed,2016-17,Willie Reed
1.0,21601205,1610612739,CLE,1627770,Kay Felder,2016-17,Kay Felder
2.0,21601205,1610612748,MIA,1626196,Josh Richardson,2016-17,Josh Richardson
3.0,21601205,1610612739,CLE,101114,Deron Williams,2016-17,Deron Williams
4.0,21601205,1610612748,MIA,201609,Goran Dragic,2016-17,Goran Dragic
...,...,...,...,...,...,...,...
NaN,21800048,1610612747,LAL,2544,LeBron James,2018-19,NaN
NaN,21800048,1610612759,SAS,200746,LaMarcus Aldridge,2018-19,NaN
NaN,21800048,1610612747,LAL,1628404,Josh Hart,2018-19,NaN


In [ ]:
import unicodedata

# Function to remove accents from text
def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalize to NFD (decomposed form) and filter out combining characters
    nfd = unicodedata.normalize('NFD', text)
    return ''.join(char for char in nfd if unicodedata.category(char) != 'Mn')

# Clean player names in final_df
df_players['PLAYER_NAME_CLEAN'] = df_players['PLAYER_NAME'].apply(remove_accents)

In [ ]:
df_players.head(1980)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,season,PLAYER_NAME_CLEAN
Unnamed: 0,,,,,,,
0.0,21601205,1610612748,MIA,203186,Willie Reed,2016-17,Willie Reed
1.0,21601205,1610612739,CLE,1627770,Kay Felder,2016-17,Kay Felder
2.0,21601205,1610612748,MIA,1626196,Josh Richardson,2016-17,Josh Richardson
3.0,21601205,1610612739,CLE,101114,Deron Williams,2016-17,Deron Williams
4.0,21601205,1610612748,MIA,201609,Goran Dragic,2016-17,Goran Dragic
...,...,...,...,...,...,...,...
NaN,21800048,1610612747,LAL,2544,LeBron James,2018-19,LeBron James
NaN,21800048,1610612759,SAS,200746,LaMarcus Aldridge,2018-19,LaMarcus Aldridge
NaN,21800048,1610612747,LAL,1628404,Josh Hart,2018-19,Josh Hart


In [ ]:
df_2016 = df[df["season"] == "2016-17"].merge(
    rapm_2016,
    left_on="PLAYER_NAME_CLEAN",
    right_on="Player",
    how="left"
)

df_2017 = df[df["season"] == "2017-18"].merge(
    rapm_2017,
    left_on="PLAYER_NAME_CLEAN",
    right_on="Player",
    how="left"
)

df_2018 = df[df["season"] == "2018-19"].merge(
    rapm_2018,
    left_on="PLAYER_NAME_CLEAN",
    right_on="Player",
    how="left"
)

df_2017.head(50)


MergeError: Passing 'suffixes' which cause duplicate columns {'ORAPM_x', 'RAPM_x', 'Team_x', 'Player_x', 'DRAPM_x', 'Poss_x'} is not allowed.